In [3]:
import pandas as pd
import numpy as np
import re
import warnings
warnings.filterwarnings("ignore")
import math
import os
import datetime
from pandas import DataFrame
import itertools
from bs4 import BeautifulSoup
import urllib.request
import requests
import random
import torch
import xmltodict
import xml.etree.ElementTree as ET
import xml
from xml.etree.ElementTree import parse
from xml.parsers import expat
from xml.dom.minidom import parse, parseString
import psycopg2
import PyPDF2
import pikepdf
from bs4 import BeautifulSoup
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
import re
import spacy

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/joythompson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/joythompson/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
def createfiledirectory(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)
    return directory

# **Define Functions to Import & Store Legislator-Committee Relationship Data from a Text File**

In [4]:
#Helper Functions to loadrepdatafromxl
def findchair(com): 
    com=com.lstrip(' ')
    if 'vice' in com:
        com=com.split(', vi')[0]
        chair='vice chair'
    elif 'chair' in com or 'chiar' in com:
        com=com.split(', ch')[0]
        chair='chair'
    else:
        chair=''
    return com, chair

def lastnamefix(lastname):
    if "'" in lastname:
        lastname=lastname.split("``")[0].rstrip(' ')
    return lastname

def longnames(name):
    if ',' in name:
        namelist=name.replace('.', '').split(',')
        name=namelist[0].replace('\n','').lstrip(' ')
        if 'of' not in namelist[1]:
            lastname=namelist[1].lstrip(' ')
            state=namelist[2].replace('\n','').split('of ')[1].lstrip(' ')
        else:
            lastname=namelist[1].replace('\n','').split(' of ')[0].lstrip(' ')
            state=namelist[1].replace('\n','').split(' of ')[1].lstrip(' ')
        lastname=lastnamefix(lastname)
    else:
        lastname=''
        state='' 
    return name, lastname, state

In [5]:
#Load Committee Representative Data from Text and Store in DataFrame
def loadrepdatafromxl(commlist):
    reppd=pd.DataFrame(columns=['Representative Name','Last Name','State', 'Committee', 'Role'])
    for nc in commlist.split('\n \n'):
        if len(nc.replace('.,','').split('...'))>1:
            name, lastname, state = longnames(list(filter(None, nc.replace('.,','').split('...')))[0].replace('\n','').lstrip(' '))
            replist=list(filter(None,list(filter(None,nc.replace('.,','').split('...')))[1].replace('  ', '').replace('\n', '').split('.')))
            for com in replist:
                com, chair=findchair(com)
                if 'Page' not in str(com):
                    reppd=reppd.append({'Representative Name': name,'Last Name':lastname,'State':state, 'Committee': com, 'Role':chair}, ignore_index=True)
        elif 'Page' not in str(nc.replace('.,','').split('...')):
            replist=list(filter(None,nc.replace('.,','').split('...')[0].replace('  ', '').replace('\n','').split('.')))
            if 'of' not in replist[0]:
                for com in replist:
                    com, chair=findchair(com)
                    reppd=reppd.append({'Representative Name': name,'Last Name':lastname,'State':state, 'Committee': com, 'Role':chair}, ignore_index=True)
            else:
                com, chair=findchair(replist[1].lstrip(' '))
                reppd=reppd.append({'Representative Name': replist[0].split(',')[0],'Last Name':lastnamefix(replist[0].split(',')[1].split(' of ')[0].lstrip(' ')),
                                    'State':replist[0].split(',')[1].split(' of ')[1],'Committee': com, 'Role':chair}, ignore_index=True)
    return reppd

# **Define Functions to Import & Store Primary Bill Data From Local XML Files**

In [ ]:
#Helper Functions to parsexml
def iterroot(root, rootstr, elements):
    detlist=[]
    if rootstr in elements:
        for var in root.iter(rootstr):
            detlist.append(var.text)
    if detlist==[]:
        detlist=['']
    return detlist

def breaknames(name, xmlnames):
    first=''
    last=''
    state=''
    if name!='':
        if 'of ' in name:
            state=name.split('of ')[1];
            fullname=name.split(' of ')[0].split(' ')[1:];
        else:
            fullname=name.split(' ')[1:];
        if len(fullname)==2:
            first=fullname[0];
            last=fullname[1];
        elif len(fullname)==3:
            first=fullname[0]+ ' ' + fullname[1][0];
            last=fullname[2];
        elif len(fullname)==4:
            first=fullname[0]+ ' ' + fullname[1][0];
            last=fullname[2] + ' '+ fullname[3];
        elif len(fullname)==1:
            last=fullname[0];
    return first, last, state

In [ ]:
#Extract Primary Bill Data from XML Files and Store in DataFrame
def parsexml(xmlnamesraw, directoryxml):

    xmlnamesraw=xmlnamesraw[['Name', 'Link', 'OfficialTitle','LegisType', 'BillStage']]
    xmlnames=xmlnamesraw.set_index('Name')
    xmlspon=pd.DataFrame(columns=[ 'BillID', 'SponsorLastName', 'SponsorFirstName', 'SponsorState', 'PrimaryOrCoSponsor'])
    xmlcomm=pd.DataFrame(columns=[ 'BillID', 'CommittteeName'])
    xmldate=pd.DataFrame(columns=[ 'BillID', 'Date'])
    
    headerpd=pd.DataFrame(columns=['BillID', 'BillName', 'Header','HeaderID', 'HeaderOrderID', 'Key', 'Text',  ])
    billid=1
    headerid=0
    for billname in xmlnames.index:
        filename=directoryxml + '/' + billname + '.xml';
        tree = ET.parse(filename);
        root = tree.getroot();
        elements=[elem.tag for elem in root.iter()];
        xmlnames.loc[billname, 'OfficialTitle']=iterroot(root, 'official-title', elements)[0];
        xmlnames.loc[billname, 'LegisType']=iterroot(root,'legis-type', elements)[0];
        
        for var in root.iter('bill'):
            xmlnames.loc[billname, 'BillStage']=var.attrib['bill-stage'];
        
        date=iterroot(root,'action-date', elements)
        commit=iterroot(root,'committee-name',elements)
        spon=iterroot(root,'sponsor',elements)
        cospon=iterroot(root,'cosponsor',elements)
        
        for idval  in spon:
            first, last, state = breaknames(idval, xmlnames);
            xmlspon=xmlspon.append({ 'BillID':billid, 'SponsorLastName':last, 'SponsorFirstName':first, 'SponsorState':state, 
                                            'PrimaryOrCoSponsor':'Primary'}, ignore_index=True);
        for idval  in cospon:
            first, last, state = breaknames(idval, xmlnames);
            xmlspon=xmlspon.append({ 'BillID':billid, 'SponsorLastName':last, 'SponsorFirstName':first, 'SponsorState':state, 
                                            'PrimaryOrCoSponsor':'Co'}, ignore_index=True);
        for idval in date:
             xmldate=xmldate.append({'BillID':billid, 'Date':idval}, ignore_index=True);
        for idval in commit:
            xmlcomm=xmlcomm.append({'BillID':billid, 'CommittteeName':idval}, ignore_index=True);
      
        lasttag=''
        headerorder=0
        for var in root.iter('*'):
            tag=var.tag;
            text=var.text;
            if tag =='header':
                headerorder=1
                headerid=headerid+1
                lasttag='header';
                header=text;
                headerpd=headerpd.append({'BillID':billid, 'BillName': billname, 'Header':header, 'HeaderID':headerid, 'HeaderOrderID':headerorder,
                                          'Key':tag, 'Text':text}, ignore_index=True);
            elif tag=='paragraph':
                lasttag='paragraph';
            elif lasttag=='header' and tag in ['text', 'quote', 'short-title', 'external-xref', 'quoted-block', 'term'] and str(text)!='None':
                headerpd=headerpd.append({'BillID':billid, 'BillName': billname, 'Header':header, 
                                           'HeaderID':headerid, 'HeaderOrderID':headerorder, 'Key':tag, 'Text':text}, ignore_index=True);
                headerorder=headerorder+1
        billid=billid+1    
    return xmlnames,  xmldate,  xmlspon, xmlcomm, headerpd

# **Call Functions to Import & Store Legislator-Committee Relationship Data from a Text File**

In [7]:
username='joythompson'
projfol="/Users/"+username+"/Desktop/DSE_203_JoyT/DSE_203_Final_Project_JoyT/"
rawdf="/Users/"+username+"/Desktop/DSE_203_JoyT/DSE_203_Final_Project_JoyT/Raw_Data_Files/"
directorybillmain=createfiledirectory(projfol+ 'Bill_Rep_Parsing_to_SQL_Main/')
directorybillhead=createfiledirectory(projfol+ 'Bill_Parsing_Headers/')

In [8]:
commlist= open(rawdf+ 'Committees.txt','r').read()
reppd= loadrepdatafromxl(commlist)
reppd.to_csv(directorybillmain +'/unique_legislator_committee_relationship_by_line.csv')

# **Call Functions to Import & Store Primary Bill Data From Local XML Files**

In [ ]:
#textnames =pd.read_csv(rawbilldf+'bill_attributes_via_text_dataframe_only_index.csv')
xmlnames = pd.read_csv(rawdf+ 'Bill_Data_from_HTML_XML/bill_attributes_via_xml_dataframe_only_index.csv')
xmlnames, xmldate,  xmlspon, xmlcomm, headerpd=parsexml(xmlnames, rawbilldf+ 'DSE_project_xml')

In [ ]:
xmlnames.to_csv(directorybillmain+ 'bill_main_attributes.csv')
xmlspon.to_csv(directorybillmain+ 'bill_sponcospon_attributes.csv')
xmldate.to_csv(directorybillmain+ 'bill_date_attributes.csv')
xmlcomm.to_csv(directorybillmain+ 'bill_committee_attributes.csv')
headerpd.to_csv(directorybillhead+ 'header_text_by_bill_header_with_order.csv')

# **Transform Stage and Sponsor Data for Neo4j**

In [6]:
def addbillstage(filestring):
    pdall=pd.read_csv(directorycleanneo+filestring)
    for ind in pdall.index:
        billid =pdall.loc[ind, 'billid']
        if billid in stagebillrel.index:
            pdall.loc[ind, 'billstage']=stagebillrel.loc[billid, 'billstage']
            pdall.loc[ind, 'chambername']=stagebillrel.loc[billid, 'chambername']
    return pdall

def definecivilians(civall, civpd):
    civall['civid']=''
    for ind in civall.index:
            last=civall.loc[ind, 'lastname']
            first=civall.loc[ind, 'firstname']
            if str(first)[0:3]!='nan' and first in civpd['firstname'].values:
                civpd=civpd[civpd['firstname']==first]

            if last in civpd['lastname'].values:
                loc=civpd['lastname']==last
                civid=civpd.loc[civpd['lastname']==last, 'civid']
                civall.loc[ind, 'civid']=civid.values[0]
    return civall

In [7]:
username='joythompson'
projfol='/Users/'+username+'/Desktop/DSE_203_JoyT/DSE_203_Final_Project_JoyT/'
directoryraw=projfol+'Raw_Data_Files/'
directorysql=projfol+'Bill_Rep_Parsing_to_SQL_Main/'
directorybillmain=projfol+ 'Bill_Parsing_Headers/'
directorycleanneo=projfol+'Final_CSV_Files_for_NEO/'
stagebillrel=pd.read_csv(directorycleanneo+ 'stage_rel_to_neo.csv').drop(columns=['link', 'officialtitle'])

civpd=pd.read_csv(directorycleanneo+'civilians_to_neo.csv').drop(columns=['state'])
repall=addbillstage('civilians_all.csv')
civall=addbillstage('repsall.csv')
civall=definecivilians(civall, civpd)
repall.to_csv(directorycleanneo+'spon_rel_data_to_neo_reps.csv')
civall.to_csv(directorycleanneo+'spon_rel_data_to_neo_civ.csv')